Let's see if we can replicate some results of 'extracting paragraphs'

In [1]:
import torch 
import math
from nnsight import CONFIG
import nnsight
import numpy as np 
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
load_dotenv()

c:\Users\Laura and Steve\Desktop\mech_interp\mech_interp_research\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
from activation_transplanting import *

Note: place ndif key and hf token in a file called .env 

eg:
```
HF_TOKEN="Token here"
NDIF_KEY="Key here"
```

In [3]:
# read the api_key
CONFIG.set_default_api_key(os.environ.get('NDIF_KEY'))

In [4]:
import os

# read the api_key

# Llama 3.1 8b is a gated model, so you need to apply for access on HuggingFace and include your token.
os.environ['HF_TOKEN'] = os.environ.get('HF_TOKEN')

## The Experiment 

Confirm the basic findings of `Extracting Paragraphs` 

Finding 1:
> Attention is focused around the `\n\n` tokens, after an instruction

Finding 2:
> By replacing the activations at newlines with another set of activations, we can make the model reproduce the original prompt 

In [5]:
from nnsight import LanguageModel
# We'll never actually load the parameters locally, so no need to specify a device_map.
llama = LanguageModel("meta-llama/Meta-Llama-3.1-8B")
# All we need to specify using NDIF vs executing locally is remote=True.

TODO: get temperature in the config 

In [6]:
num_examples = 1 # how many examples to use in the context?

with open("context.txt", 'r') as file:
    context = file.read()

context = "\n--\n".join(context.split("\n--\n")[:num_examples])+"\n--\n"

In [37]:
print(context)

What are the key elements of photosynthesis?

Photosynthesis is the process where plants convert sunlight, water, and carbon dioxide into glucose and oxygen. It occurs in chloroplasts containing chlorophyll, which captures light energy. The process includes light-dependent reactions producing ATP and NADPH, and the Calvin cycle that uses these to fix carbon dioxide into sugar.
--



In [23]:
s1 = "What was the cause of the Punic War?\n\n"
s2 = "What were the causes of the Civil War?\n\n"


In [24]:
print(context+s1)

What are the key elements of photosynthesis?

Photosynthesis is the process where plants convert sunlight, water, and carbon dioxide into glucose and oxygen. It occurs in chloroplasts containing chlorophyll, which captures light energy. The process includes light-dependent reactions producing ATP and NADPH, and the Calvin cycle that uses these to fix carbon dioxide into sugar.
--
What was the cause of the Punic War?




In [28]:
tk = LLamaExamineToolkit(
    llama_model=llama, 
    remote=True, # use NDIF
    num_prev=1 # Take one additional token backwards in time -- not only the ?/n/n token, but one extra
)

In [36]:
outputs = tk.transplant_newline_activities(
    source_strings=[context+s1], # add context to the specific question we'll use as the source
    target_strings=[context+s2], # add contect to the question we'll use as the target
    num_new_tokens=50, # how many new tokens to generate
    index=-1, # swap at the last instance of ?/n/n
    transplant_strings = ("key", "value") # swap "key" and "value" tensors
)

extracting newline activations


2025-03-03 22:30:42,743 ab8c1ff5-c597-4e20-8972-47c75206fdd7 - RECEIVED: Your job has been received and is waiting approval.
2025-03-03 22:30:43,066 ab8c1ff5-c597-4e20-8972-47c75206fdd7 - APPROVED: Your job was approved and is waiting to be run.
2025-03-03 22:30:43,441 ab8c1ff5-c597-4e20-8972-47c75206fdd7 - RUNNING: Your job has started running.
2025-03-03 22:30:45,013 ab8c1ff5-c597-4e20-8972-47c75206fdd7 - COMPLETED: Your job has been completed.


source_token =  82 ('?\n\n', 1980)
target_token =  82 ?


source_token =  81 (' War', 5111)
target_token =  81  War
source_token =  82 ('?\n\n', 1980)
target_token =  82 ?


source_token =  81 (' War', 5111)
target_token =  81  War
source_token =  82 ('?\n\n', 1980)
target_token =  82 ?


source_token =  81 (' War', 5111)
target_token =  81  War
source_token =  82 ('?\n\n', 1980)
target_token =  82 ?


source_token =  81 (' War', 5111)
target_token =  81  War
source_token =  82 ('?\n\n', 1980)
target_token =  82 ?


source_token =  81 (' War', 5111)
target_token =  81  War
source_token =  82 ('?\n\n', 1980)
target_token =  82 ?


source_token =  81 (' War', 5111)
target_token =  81  War
source_token =  82 ('?\n\n', 1980)
target_token =  82 ?


source_token =  81 (' War', 5111)
target_token =  81  War
source_token =  82 ('?\n\n', 1980)
target_token =  82 ?


source_token =  81 (' War', 5111)
target_token =  81  War
source_token =  82 ('?\n\n', 1980)
target_token =  82 ?


source_token =

2025-03-03 22:30:47,984 21907806-f7b5-4fb8-86d7-9257ecbe9d90 - RECEIVED: Your job has been received and is waiting approval.
2025-03-03 22:30:48,818 21907806-f7b5-4fb8-86d7-9257ecbe9d90 - APPROVED: Your job was approved and is waiting to be run.
2025-03-03 22:30:49,443 21907806-f7b5-4fb8-86d7-9257ecbe9d90 - RUNNING: Your job has started running.
2025-03-03 22:30:51,624 21907806-f7b5-4fb8-86d7-9257ecbe9d90 - COMPLETED: Your job has been completed.


In [35]:
for out in outputs:
    print(out)
    print("--------------")

<|begin_of_text|>What are the key elements of photosynthesis?

Photosynthesis is the process where plants convert sunlight, water, and carbon dioxide into glucose and oxygen. It occurs in chloroplasts containing chlorophyll, which captures light energy. The process includes light-dependent reactions producing ATP and NADPH, and the Calvin cycle that uses these to fix carbon dioxide into sugar.
--
What were the causes of the Civil War?

The cause of the First Punic War was the struggle for control of the Western Mediterranean. Carthage was the dominant power in the region, but Rome began to expand its territory. The two powers fought for control of Sicily, and Carth
--------------
